In [1]:
import json
import joblib

from pathlib import Path

import pandas as pd
import numpy as np

from isic_helper import DotDict, get_folds

In [2]:
cfg = DotDict()

cfg.models_output_dir = "models"
cfg.model_name = "boosting_v1"

In [3]:
id_column = 'isic_id'
target_column = 'target'
group_column = 'patient_id'

numerical_features = [
    "age_approx",
    "clin_size_long_diam_mm",
    "tbp_lv_A", "tbp_lv_Aext",
    "tbp_lv_B", "tbp_lv_Bext",
    "tbp_lv_C", "tbp_lv_Cext",
    "tbp_lv_H", "tbp_lv_Hext",
    "tbp_lv_L", "tbp_lv_Lext",
    "tbp_lv_areaMM2",
    "tbp_lv_area_perim_ratio",
    "tbp_lv_color_std_mean",
    "tbp_lv_deltaA", "tbp_lv_deltaB", "tbp_lv_deltaL", "tbp_lv_deltaLB", "tbp_lv_deltaLBnorm",
    "tbp_lv_eccentricity",
    "tbp_lv_minorAxisMM",
    "tbp_lv_nevi_confidence",
    "tbp_lv_norm_border", "tbp_lv_norm_color",
    "tbp_lv_perimeterMM",
    "tbp_lv_radial_color_std_max",
    "tbp_lv_stdL", "tbp_lv_stdLExt",
    "tbp_lv_symm_2axis", "tbp_lv_symm_2axis_angle",
    "tbp_lv_x", "tbp_lv_y", "tbp_lv_z",
]

ord_categorical_features = []

ohe_categorical_features = [
    "sex", 
    "anatom_site_general", 
    "tbp_tile_type", 
    "tbp_lv_location", 
    "tbp_lv_location_simple",
    "attribution",
]

def preprocess(df):
    df["anatom_site_general"] = df["anatom_site_general"].fillna("missing")
    df["sex"] = df["sex"].fillna("missing")
    df["tbp_tile_type"] = df["tbp_tile_type"].map({"3D: white": "white", "3D: XP": "XP"})
    df["anatom_site_general"] = df["anatom_site_general"].str.replace(" ", "-").str.replace("/", "OR")
    df["tbp_lv_location"] = df["tbp_lv_location"].str.replace(" ", "").str.replace("&", "AND")
    df["tbp_lv_location_simple"] = df["tbp_lv_location_simple"].str.replace(" ", "").str.replace("&", "AND")
    attribution_mapper = {
        "Memorial Sloan Kettering Cancer Center": "MSKCC",
        "ACEMID MIA": "ACEMIDMIA",
        "Department of Dermatology, Hospital Clínic de Barcelona": "DoD_HCB",
        "University Hospital of Basel": "UHB",
        "Frazer Institute, The University of Queensland, Dermatology Research Centre": "FI_TUQ-DRC",
        "Department of Dermatology, University of Athens, Andreas Syggros Hospital of Skin and Venereal Diseases, Alexander Stratigos, Konstantinos Liopyris": "DoD_UA",
        "ViDIR Group, Department of Dermatology, Medical University of Vienna": "ViDIR"
    }
    df["attribution"] = df["attribution"].map(attribution_mapper)
    
    return df

def stat_feature(df, value_col, stats=["mean", "std"], err=1e-5):
    tmp = df.groupby(group_column)[value_col].agg(stats)
    tmp.columns = [f"{value_col}_{stat}" for stat in stats]
    tmp.reset_index(inplace=True)
    df = df.merge(tmp, on=group_column, how="left")
    feature_name = f"{value_col}_patient_norm"
    df[feature_name] = ((df[value_col] - df[f"{value_col}_mean"]) / 
                                       (df[f"{value_col}_std"] + err))
    return df, feature_name

def feature_engineering(df, err=1e-5):
    new_num_cols = []
    
    df["lesion_size_ratio"] = df["tbp_lv_minorAxisMM"] / df["clin_size_long_diam_mm"]
    new_num_cols += ["lesion_size_ratio"]
    
    df["lesion_shape_index"] = df["tbp_lv_areaMM2"] / df["tbp_lv_perimeterMM"]**2
    new_num_cols += ["lesion_shape_index"]
    
    df["hue_contrast"] = np.abs(df["tbp_lv_H"] - df["tbp_lv_Hext"])
    new_num_cols += ["hue_contrast"]
    
    df["luminance_contrast"] = np.abs(df["tbp_lv_L"] - df["tbp_lv_Lext"])
    new_num_cols += ["luminance_contrast"]
    
    df["lesion_color_difference"] = np.sqrt(df["tbp_lv_deltaA"]**2 +
                                            df["tbp_lv_deltaB"]**2 +
                                            df["tbp_lv_deltaL"]**2)
    new_num_cols += ["lesion_color_difference"]
    
    df["border_complexity"] = df["tbp_lv_norm_border"] + df["tbp_lv_symm_2axis"]
    new_num_cols += ["border_complexity"]
    
    df["color_uniformity"] = df["tbp_lv_color_std_mean"] / (df["tbp_lv_radial_color_std_max"] + err)
    new_num_cols += ["color_uniformity"]
    
    df["position_distance_3d"] = np.sqrt(df["tbp_lv_x"]**2 +
                                         df["tbp_lv_y"]**2 +
                                         df["tbp_lv_z"]**2)
    new_num_cols += ["position_distance_3d"]
    
    df["perimeter_to_area_ratio"] = df["tbp_lv_perimeterMM"] / df["tbp_lv_areaMM2"]
    new_num_cols += ["perimeter_to_area_ratio"]
    
    df["area_to_perimeter_ratio"] = df["tbp_lv_areaMM2"] / df["tbp_lv_perimeterMM"]
    new_num_cols += ["area_to_perimeter_ratio"]
    
    df["lesion_visibility_score"] = df["tbp_lv_deltaLBnorm"] + df["tbp_lv_norm_color"]
    new_num_cols += ["lesion_visibility_score"]
    
    df["symmetry_border_consistency"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_norm_border"]
    new_num_cols += ["symmetry_border_consistency"]
    
    df["consistency_symmetry_border"] = (df["tbp_lv_symm_2axis"] * df["tbp_lv_norm_border"] /
                                         (df["tbp_lv_symm_2axis"] + df["tbp_lv_norm_border"]))
    new_num_cols += ["consistency_symmetry_border"]
    
    df["color_consistency"] = df["tbp_lv_stdL"] / df["tbp_lv_Lext"]
    new_num_cols += ["color_consistency"]
    
    df["consistency_color"] = (df["tbp_lv_stdL"] * df["tbp_lv_Lext"] /
                               (df["tbp_lv_stdL"] * df["tbp_lv_Lext"]))
    new_num_cols += ["consistency_color"]
    
    df["size_age_interaction"] = df["clin_size_long_diam_mm"] * df["age_approx"]
    new_num_cols += ["size_age_interaction"]
    
    df["hue_color_std_interaction"] = df["tbp_lv_H"] * df["tbp_lv_color_std_mean"]
    new_num_cols += ["hue_color_std_interaction"]
    
    df["lesion_severity_index"] = (df["tbp_lv_norm_border"] +
                                   df["tbp_lv_norm_color"] +
                                   df["tbp_lv_eccentricity"]) / 3
    new_num_cols += ["lesion_severity_index"]
    
    df["shape_complexity_index"] = df["border_complexity"] + df["lesion_shape_index"]
    new_num_cols += ["shape_complexity_index"]
    
    df["color_contrast_index"] = (df["tbp_lv_deltaA"] +
                                  df["tbp_lv_deltaB"] + 
                                  df["tbp_lv_deltaL"] +
                                  df["tbp_lv_deltaLBnorm"])
    new_num_cols += ["color_contrast_index"]
    
    df["log_lesion_area"] = np.log1p(df["tbp_lv_areaMM2"])
    new_num_cols += ["log_lesion_area"]
    
    df["normalized_lesion_size"] = df["clin_size_long_diam_mm"] / df["age_approx"]
    new_num_cols += ["normalized_lesion_size"]
    
    df["mean_hue_difference"] = (df["tbp_lv_H"] + df["tbp_lv_Hext"]) / 2
    new_num_cols += ["mean_hue_difference"]
    
    df["std_dev_contrast"] = np.sqrt((df["tbp_lv_deltaA"]**2 +
                                      df["tbp_lv_deltaB"]**2 + 
                                      df["tbp_lv_deltaL"]**2) / 3)
    new_num_cols += ["std_dev_contrast"]
    
    df["color_shape_composite_index"] = (df["tbp_lv_color_std_mean"] + 
                                         df["tbp_lv_area_perim_ratio"] +
                                         df["tbp_lv_symm_2axis"]) / 3
    new_num_cols += ["color_shape_composite_index"]
    
    df["lesion_orientation_3d"] = np.arctan2(df["tbp_lv_y"], df["tbp_lv_x"])
    new_num_cols += ["lesion_orientation_3d"]
    
    df["overall_color_difference"] = (df["tbp_lv_deltaA"] + 
                                      df["tbp_lv_deltaB"] + 
                                      df["tbp_lv_deltaL"]) / 3
    new_num_cols += ["overall_color_difference"]
    
    df["symmetry_perimeter_interaction"] = df["tbp_lv_symm_2axis"] * df["tbp_lv_perimeterMM"]
    new_num_cols += ["symmetry_perimeter_interaction"]
    
    df["comprehensive_lesion_index"] = (df["tbp_lv_area_perim_ratio"] +
                                        df["tbp_lv_eccentricity"] +
                                        df["tbp_lv_norm_color"] +
                                        df["tbp_lv_symm_2axis"]) / 4
    new_num_cols += ["comprehensive_lesion_index"]
    
    df["color_variance_ratio"] = df["tbp_lv_color_std_mean"] / df["tbp_lv_stdLExt"]
    new_num_cols += ["color_variance_ratio"]
    
    df["border_color_interaction"] = df["tbp_lv_norm_border"] * df["tbp_lv_norm_color"]
    new_num_cols += ["border_color_interaction"]
    
    df["border_color_interaction_2"] = ((df["tbp_lv_norm_border"] * df["tbp_lv_norm_color"]) /
                                        (df["tbp_lv_norm_border"] + df["tbp_lv_norm_color"]))
    new_num_cols += ["border_color_interaction_2"]
    
    df["size_color_contrast_ratio"] = df["clin_size_long_diam_mm"] / df["tbp_lv_deltaLBnorm"]
    new_num_cols += ["size_color_contrast_ratio"]
    
    df["age_normalized_nevi_confidence"] = df["tbp_lv_nevi_confidence"] / df["age_approx"]
    new_num_cols += ["age_normalized_nevi_confidence"]
    
    df["age_normalized_nevi_confidence_2"] = np.sqrt(df["tbp_lv_nevi_confidence"]**2 + df["age_approx"]**2)
    new_num_cols += ["age_normalized_nevi_confidence_2"]
    
    df["color_asymmetry_index"] = df["tbp_lv_radial_color_std_max"] * df["tbp_lv_symm_2axis"]
    new_num_cols += ["color_asymmetry_index"]
    
    df["volume_approximation_3d"] = df["tbp_lv_areaMM2"] * np.sqrt(df["tbp_lv_x"]**2 +
                                                                   df["tbp_lv_y"]**2 +
                                                                   df["tbp_lv_z"]**2)
    new_num_cols += ["volume_approximation_3d"]
    
    df["color_range"] = (np.abs(df["tbp_lv_L"] - df["tbp_lv_Lext"]) +
                         np.abs(df["tbp_lv_A"] - df["tbp_lv_Aext"]) +
                         np.abs(df["tbp_lv_B"] - df["tbp_lv_Bext"]))
    new_num_cols += ["color_range"]
    
    df["shape_color_consistency"] = df["tbp_lv_eccentricity"] * df["tbp_lv_color_std_mean"]
    new_num_cols += ["shape_color_consistency"]
    
    df["border_length_ratio"] = df["tbp_lv_perimeterMM"] / np.sqrt(2 * df["tbp_lv_areaMM2"])
    new_num_cols += ["border_length_ratio"]
    
    df["age_size_symmetry_index"] = (df["age_approx"] *
                                     df["clin_size_long_diam_mm"] *
                                     df["tbp_lv_symm_2axis"])
    new_num_cols += ["age_size_symmetry_index"]
    
    df["age_area_symmetry"] = (df["age_approx"] *
                               df["tbp_lv_areaMM2"] *
                               df["tbp_lv_symm_2axis"])
    new_num_cols += ["age_area_symmetry"]
    
    for col in numerical_features + new_num_cols:
        df, feature_name = stat_feature(df, col)
        new_num_cols += [feature_name]
    
    df["num_images"] = df.patient_id.map(df.groupby(group_column)[id_column].count())
    new_num_cols += ["num_images"]

    return df, new_num_cols

In [4]:
INPUT_PATH = Path("/kaggle/input/isic-2024-challenge/")
ARTIFACTS_INPUT_PATH = Path("/kaggle/input/isic-scd-boosting-v1-train/")
MODELS_INPUT_PATH = ARTIFACTS_INPUT_PATH / cfg.models_output_dir

train_metadata = pd.read_csv(INPUT_PATH / "train-metadata.csv", low_memory=False, na_values=["NA"])
test_metadata = pd.read_csv(INPUT_PATH / "test-metadata.csv", low_memory=False, na_values=["NA"])

folds_df = get_folds()
train_metadata = train_metadata.merge(folds_df, on=["isic_id", "patient_id"], how="inner")
print(f"Train data size: {train_metadata.shape}")
print(f"Test data size: {test_metadata.shape}")

train_metadata = preprocess(train_metadata)
test_metadata = preprocess(test_metadata)

train_metadata, new_num_cols = feature_engineering(train_metadata)
test_metadata, _ = feature_engineering(test_metadata)

Train data size: (401059, 57)
Test data size: (3, 44)


In [5]:
with open(ARTIFACTS_INPUT_PATH / f"{cfg.model_name}_encoder.joblib", "rb") as f:
    mixed_encoded_preprocessor = joblib.load(f)
    
enc = mixed_encoded_preprocessor.fit(train_metadata)
X_test = enc.transform(test_metadata)

columns_for_model = len(X_test.columns)
print(f"Total number of columns: {columns_for_model}")

Total number of columns: 200


In [6]:
all_folds = np.unique(train_metadata["fold"])
test_predictions_df = pd.DataFrame({id_column: test_metadata[id_column]})
for fold in all_folds:
    with open(MODELS_INPUT_PATH / f"{cfg.model_name}_fold_{fold}.txt", "rb") as f:
        estimator = joblib.load(f)
    test_predictions_df[f"fold_{fold}"] = estimator.predict_proba(X_test)[:, -1]

In [7]:
test_predictions_df.head()

,isic_id,fold_1,fold_2,fold_3,fold_4,fold_5
0,ISIC_0015657,0.521433,0.540319,0.046139,0.661866,0.519844
1,ISIC_0015729,0.503948,0.523280,0.044461,0.672395,0.468169
2,ISIC_0015740,0.577508,0.592682,0.056244,0.700100,0.610509


In [8]:
test_predictions_df[target_column] = test_predictions_df[[f"fold_{fold}" for fold in all_folds]].mean(axis=1)

In [9]:
test_predictions_df.head()

,isic_id,fold_1,fold_2,fold_3,fold_4,fold_5,target
0,ISIC_0015657,0.521433,0.540319,0.046139,0.661866,0.519844,0.457920
1,ISIC_0015729,0.503948,0.523280,0.044461,0.672395,0.468169,0.442451
2,ISIC_0015740,0.577508,0.592682,0.056244,0.700100,0.610509,0.507409


In [10]:
test_predictions_df[target_column].describe()

count    3.000000
mean     0.469260
std      0.033931
min      0.442451
25%      0.450186
50%      0.457920
75%      0.482664
max      0.507409
Name: target, dtype: float64

In [11]:
test_predictions_df[[id_column, target_column]].to_csv("submission.csv", index=False)